In [1]:
import pandas as pd
import re

def process_text(input_text):
    # Добавляем пробелы к букве "R", если она идет слитно с цифрой
    # pattern1 = r'(R)(\d+)'
    # processed_text = re.sub(pattern1, r'R \2', input_text)

    # pattern2 = r'(\d+)(R)'
    # processed_text = re.sub(pattern2, r'\1 R', processed_text)

    pattern3 = r'(\/)(\d+)'
    processed_text = re.sub(pattern3, r'/ \2', input_text)

    pattern4 = r'(\d+)(\/)'
    processed_text = re.sub(pattern4, r'\1 /', processed_text)

    pattern5 = r'(\\)(\d+)'
    processed_text = re.sub(pattern5, r'\\ \2', processed_text)

    pattern6 = r'(\d+)(\\)'
    processed_text = re.sub(pattern6, r'\1 \\', processed_text)

    # Заменяем символы "/", "\", "|" на пробелы
    # processed_text = re.sub(r'[\/\\|]', ' ', processed_text)

    # Заменяем символы "/" на " / "
    # processed_text = processed_text.replace("/", " / ")
    # Заменяем символы "\" на " \ "
    # processed_text = processed_text.replace("\\", " \\ ")
    # Заменяем символы "|" на " | "
    processed_text = processed_text.replace("|", " | ")

    # Убираем повторяющиеся пробелы
    processed_text = re.sub(r'  +', ' ', processed_text)
    return processed_text

def create_bio_tags(row):
    name = row['offer']
    width = str(row['width'])
    height = str(row['height'])
    radius = str(row['radius'])
    brand = row['brand']
    line = str(row['line'])
    
    # Создаем пустой список для хранения тегов
    tags = ['O'] * len(name.split())
    Begin_line = True
    # Проверяем каждое слово в наименовании и устанавливаем соответствующий тег
    for i, word in enumerate(name.split()):
        if word == '/':
            continue
        if word == 'R':
            continue
        if word == width:
            tags[i] = 'B-width'
        elif word == height:
            tags[i] = 'B-height'
        elif word == radius:
            tags[i] = 'B-radius'
        elif word in brand:
            tags[i] = 'B-brand'
        elif word in line:
            if Begin_line:
                tags[i] = 'B-line'
                Begin_line = False
            else:
                tags[i] = 'I-line'
            
    # Собираем теги в строку
    return ','.join(tags)

Наши данные

In [2]:
pth = '/home/sondors/Documents/price/BERT_NER/csv/Kazakov_241123/Continental_character.xlsx'   
df = pd.read_excel(pth, dtype=str)
df
# print(df.keys())

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Комментарии,Наименование,Модель,Ширина шины,Высота профиля,...,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
0,96794,NaN,8101,NaN,7317,NaN,[CONTISPORTCONTACT 5 SUV 23555R19 105W XL FR L...,SportContact 5 SUV,235,55,...,NaN,NaN,Auto,Все товары :: Авто :: Шины и диски :: Шины,Continental,https://avatars.mds.yandex.net/get-mpic/527704...,[8101],2023-07-29 18:50:34,NaN,1768586022
1,61852,price,8101,NaN,7317,NaN,125/80 R13 Continental ContiWinterContact TS80...,WinterContact TS800,125,80,...,355286,NaN,Auto,Шины,Continental,https://euro-diski.ru/upload/iblock/dd6/a72e54...,"[8101, 1403, 350201]",2023-08-09 14:10:08,NaN,1777012756
2,96794,NaN,8101,NaN,7317,NaN,14/155/65 Continental IceContact 3 TA 75T,IceContact 3,155,65,...,NaN,NaN,Auto,Все товары :: Авто :: Шины и диски :: Шины,Continental,https://avatars.mds.yandex.net/get-mpic/449302...,[8101],2023-07-29 18:50:34,NaN,1754590109
3,61852,NaN,8101,NaN,7317,NaN,14/185/60 Continental IceContact 3 TA 82T,IceContact 3,185,60,...,347594,NaN,Auto,Шины,Continental,https://euro-diski.ru/upload/iblock/72a/cf125c...,[8101],2023-07-07 14:50:12,NaN,1755712962
4,96794,NaN,8101,NaN,7317,NaN,145/65 R15 72T Continental VikingContact 7,VikingContact 7,145,65,...,NaN,NaN,Auto,Все товары :: Авто :: Шины и диски :: Шины,Continental,https://avatars.mds.yandex.net/get-mpic/136080...,"[8101, 829805]",2023-05-05 05:06:00,NaN,1694032869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20276,96794,NaN,8101,NaN,7317,NaN,Континенталь 295/30/19 W 100 ContiWinterContac...,WinterContact TS830,295,30,...,NaN,NaN,Auto,Все товары :: Авто :: Шины и диски :: Шины,Continental,https://avatars.mds.yandex.net/get-mpic/408043...,"[8101, 2110, 7502, 5999]",2023-05-02 11:20:06,NaN,1694349152
20277,96794,NaN,8101,NaN,7317,NaN,Континенталь 295/30/21 V 102 WinterContact TS ...,WinterContact TS860,295,30,...,NaN,NaN,Auto,Все товары :: Авто :: Шины и диски :: Шины,Continental,https://avatars.mds.yandex.net/get-mpic/765032...,"[8101, 7502]",2023-05-02 17:08:57,NaN,1695507907
20278,96794,NaN,8101,NaN,7317,NaN,Континенталь 295/40/19 V 108 ContiWinterContac...,WinterContact TS830,295,40,...,NaN,NaN,Auto,Все товары :: Авто :: Шины и диски :: Шины,Continental,https://avatars.mds.yandex.net/get-mpic/408043...,[8101],2023-06-08 16:53:29,NaN,1694665532
20279,96794,NaN,8101,NaN,7317,NaN,Континенталь 305/40/20 Y 112 ContiSportContact...,SportContact 5,305,40,...,NaN,NaN,Auto,Все товары :: Авто :: Шины и диски :: Шины,Continental,https://avatars.mds.yandex.net/get-mpic/405568...,"[8101, 8398, 719825]",2023-05-02 17:08:57,NaN,1695320371


In [3]:
def change_columns(df):   
    new_df = pd.DataFrame()

    df = df.rename(columns={'Наименование': 'offer'})
    df = df[df['offer'].notna()]

    new_df['offer'] = df['offer']
    new_df['width'] = df['Ширина шины']
    new_df['height'] = df['Высота профиля']
    new_df['radius'] = df['Диаметр']
    new_df['v_ind'] = df['Индекс нагрузки и скорости']
    new_df['brand'] = df['Unnamed: 29']
    new_df['line'] = df['Модель']

    new_df['offer'] = new_df['offer'].apply(process_text)
    new_df['brand'] = new_df['offer'].apply(extract_Continental)
    new_df['BIO_Tags'] = new_df.apply(create_bio_tags, axis=1)
    return new_df

def create_bio_tags(row):
    name = str(row['offer'])
    width = str(row['width'])
    height = str(row['height'])
    radius = str(row['radius'])
    brand = row['brand']
    line = str(row['line'])
    v_ind = str(row['v_ind'])

    # Создаем пустой список для хранения тегов
    tags = ['O'] * len(name.split())
    Begin_line = True
    Begin_v_ind = True
    Begin_radius = True
    # Проверяем каждое слово в наименовании и устанавливаем соответствующий тег
    for i, word in enumerate(name.split()):
        if word == '/':
            continue
        # if word == 'R':
        #     continue
        if word == width:
            tags[i] = 'B-width'
        elif word == height:
            tags[i] = 'B-height'
        elif word.upper() in radius.upper():
            if Begin_radius:
                tags[i] = 'B-radius'
                Begin_radius = False
            else:
                tags[i] = 'I-radius'
        elif word in brand:
            tags[i] = 'B-brand'
        elif word in line:
            if Begin_line:
                tags[i] = 'B-line'
                Begin_line = False
            else:
                tags[i] = 'I-line'
        elif word.upper() in v_ind.upper():
            if Begin_v_ind:
                tags[i] = 'B-v_ind'
                Begin_v_ind = False
            else:
                tags[i] = 'I-v_ind'    

        elif word.startswith('Conti') and word[5:] in line:    #   для Continental добавим искусственно
            if Begin_line:
                tags[i] = 'B-line'
                Begin_line = False
            else:
                tags[i] = 'I-line'

    # Собираем теги в строку
    return ','.join(tags)

def extract_Bridgestone(text):
    brands = re.findall(r'Bridgestone|BRIDGESTONE|Бриджстоун|bridgestone|Бриджстон|Bridgeston|бриджстоун|бриджстон', text)
    return brands

def extract_Continental(text):
    brands = re.findall(r'Continental|CONTINENTAL|Континенталь|continental|КОНТИНЕНТАЛЬ', text)
    return brands

new_df = change_columns(df)
new_df

,offer,width,height,radius,v_ind,brand,line,BIO_Tags
0,[CONTISPORTCONTACT 5 SUV 23555R19 105W XL FR L...,235,55,R19,105W,[CONTINENTAL],SportContact 5 SUV,"O,B-line,I-line,O,B-v_ind,O,O,O,O,B-brand"
1,125 / 80 R13 Continental ContiWinterContact TS...,125,80,R13,65Q,[Continental],WinterContact TS800,"B-width,O,B-height,B-radius,B-brand,B-line,I-l..."
2,14 / 155 / 65 Continental IceContact 3 TA 75T,155,65,R14,75T,[Continental],IceContact 3,"B-radius,O,B-width,O,B-height,B-brand,B-line,I..."
3,14 / 185 / 60 Continental IceContact 3 TA 82T,185,60,R14,82T,[Continental],IceContact 3,"B-radius,O,B-width,O,B-height,B-brand,B-line,I..."
4,145 / 65 R15 72T Continental VikingContact 7,145,65,R15,72T,[Continental],VikingContact 7,"B-width,O,B-height,B-radius,B-v_ind,B-brand,B-..."
...,...,...,...,...,...,...,...,...
20276,Континенталь 295 / 30 / 19 W 100 ContiWinterCo...,295,30,R19,100W,[Континенталь],WinterContact TS830,"B-brand,B-width,O,B-height,O,B-radius,B-line,B..."
20277,Континенталь 295 / 30 / 21 V 102 WinterContact...,295,30,R21,102V,[Континенталь],WinterContact TS860,"B-brand,B-width,O,B-height,O,B-radius,B-v_ind,..."
20278,Континенталь 295 / 40 / 19 V 108 ContiWinterCo...,295,40,R19,108V,[Континенталь],WinterContact TS830,"B-brand,B-width,O,B-height,O,B-radius,B-v_ind,..."
20279,Континенталь 305 / 40 / 20 Y 112 ContiSportCon...,305,40,R20,112Y,[Континенталь],SportContact 5,"B-brand,B-width,O,B-height,O,B-radius,B-v_ind,..."


In [10]:
index = 20279
example = dict(new_df.iloc[index])
print(example['offer'],'\n')
offer = example['offer'].split()
BIO_Tags = example['BIO_Tags'].split(',')

for i in range(len(offer)):
    print(f"{offer[i]}: {BIO_Tags[i]}")

Континенталь 305 / 40 / 20 Y 112 ContiSportContact 5P XL 

Континенталь: B-brand
305: B-width
/: O
40: B-height
/: O
20: B-radius
Y: B-v_ind
112: I-v_ind
ContiSportContact: B-line
5P: O
XL: O


In [23]:
ilocc = df.iloc[8273]
print(ilocc.offer, '\n')
BIO = ilocc.BIO_Tags.split(',')
ofer = ilocc.offer.split()
for i in range(len(BIO)):
    print(f"{ofer[i]} --> {BIO[i]}")


R 19 225 / 45 92W Bridgestone Dueler H/P Sport (уценка) BRIDGESTONE 9776 

R --> O
19 --> B-radius
225 --> B-width
/ --> O
45 --> B-height
92W --> O
Bridgestone --> B-brand
Dueler --> B-line
H/P --> I-line
Sport --> I-line
(уценка) --> O
BRIDGESTONE --> B-brand
9776 --> O


Опенсорс

In [24]:
def clear_df(df):
    df = df[df['offer'].notna()]
    df = df[df['brand'].notna()]
    df = df[df['width'].notna()]
    df = df[df['height'].notna()]
    df = df[df['radius'].notna()]
    df = df[df.apply(lambda row: str(row['width']) in row['offer'], axis=1)]
    df = df[df.apply(lambda row: str(row['height']) in row['offer'], axis=1)]
    df = df[df.apply(lambda row: str(row['radius']) in row['offer'], axis=1)]
    return df.reset_index(drop=True)
    
pth = '/home/sondors/Documents/price/BERT_NER/opensource.csv'
clms = ['width_indices', 'height_indices', 'radius_indices', 'brand_indices',
       'line_indices']
    
df_opensource = pd.read_csv(pth, sep=';')
df_opensource = df_opensource.drop(columns=clms)

df_opensource = clear_df(df_opensource)
df_opensource['offer'] = df_opensource['offer'].apply(process_text)

# Применяем функцию к каждой строке DataFrame и создаем новую колонку с тегами
df_opensource['BIO_Tags'] = df_opensource.apply(create_bio_tags, axis=1)

df_opensource


,width,height,radius,brand,offer,type,line,BIO_Tags
0,175,65,14,Amtel,Amtel 175 / 65 R 14 82Q Nordmaster (В-228),Колобокс_Шины,NaN,"B-brand,B-width,O,B-height,O,B-radius,O,O,O"
1,175,70,13,Avatyre,Avatyre 175 / 70 R 13 82Q Freeze,Колобокс_Шины,NaN,"B-brand,B-width,O,B-height,O,B-radius,O,O"
2,205,55,16,Avatyre,Avatyre 205 / 55 R 16 91T Freeze,Колобокс_Шины,NaN,"B-brand,B-width,O,B-height,O,B-radius,O,O"
3,215,65,16,Avatyre,Avatyre 215 / 65 R 16 98T Freeze,Колобокс_Шины,NaN,"B-brand,B-width,O,B-height,O,B-radius,O,O"
4,215,65,16,Avatyre,Avatyre 215 / 65 R 16 98T Freeze Товар с уценкой,Колобокс_Шины,NaN,"B-brand,B-width,O,B-height,O,B-radius,O,O,O,O,O"
...,...,...,...,...,...,...,...,...
5865,425,85,21,НК.ШЗ,425 / 85 R 21 КАМА-1260-1 НК.ШЗ 14 146 J,грузовая_резина,КАМА-1260-1,"B-width,O,B-height,O,B-radius,B-line,B-brand,O..."
5866,425,85,21,НК.ШЗ,425 / 85 R 21 КАМА-1260-2 НК.ШЗ 18,грузовая_резина,КАМА-1260-2,"B-width,O,B-height,O,B-radius,B-line,B-brand,O"
5867,425,85,21,Омск.ШЗ,425 / 85 R 21 О-184 TYREX CRG POWER Омск.ШЗ 14...,грузовая_резина,О-184 TYREX CRG POWER,"B-width,O,B-height,O,B-radius,B-line,I-line,I-..."
5868,425,85,21,Омск.ШЗ,425 / 85 R 21 О-184 TYREX CRG POWER Омск.ШЗ 18...,грузовая_резина,О-184 TYREX CRG POWER,"B-width,O,B-height,O,B-radius,B-line,I-line,I-..."


In [25]:
ilocc.radius

'19'

In [26]:
ilocc = df_opensource.iloc[5867]
print(ilocc.offer, '\n')
BIO = ilocc.BIO_Tags.split(',')
ofer = ilocc.offer.split()
for i in range(len(BIO)):
    print(f"{ofer[i]} --> {BIO[i]}")


425 / 85 R 21 О-184 TYREX CRG POWER Омск.ШЗ 14 146 K 

425 --> B-width
/ --> O
85 --> B-height
R --> O
21 --> B-radius
О-184 --> B-line
TYREX --> I-line
CRG --> I-line
POWER --> I-line
Омск.ШЗ --> B-brand
14 --> O
146 --> O
K --> O


Train/test split, concat

In [27]:
train_dataset = df.sample(frac=0.8,random_state=200)
test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

In [28]:
test_dataset

,offer,width,height,radius,v_ind,line,brand,type,BIO_Tags
0,14 / 165 / 65 Bridgestone Blizzak LM005 79T,165,65,16,79T,Blizzak LM005,[Bridgestone],наши_данные,"O,O,B-width,O,B-height,B-brand,B-line,I-line,O"
1,15 / 175 / 65 Bridgestone Turanza T005 84H,175,65,15,84H,Turanza T005,[Bridgestone],наши_данные,"B-radius,O,B-width,O,B-height,B-brand,B-line,I..."
2,15 / 185 / 55 Bridgestone Turanza T005 82V,185,55,18,82V,Turanza T005,[Bridgestone],наши_данные,"O,O,B-width,O,B-height,B-brand,B-line,I-line,O"
3,15 / 205 / 70 Bridgestone Blizzak LM005 96T,205,70,15,96T,Blizzak LM005,[Bridgestone],наши_данные,"B-radius,O,B-width,O,B-height,B-brand,B-line,I..."
4,15169 BRIDGESTONE Автошина R 15 185 / 60 Bridg...,185,60,15,88T,Blizzak LM005,"[BRIDGESTONE, Bridgestone]",наши_данные,"O,B-brand,O,O,B-radius,B-width,O,B-height,B-br..."
...,...,...,...,...,...,...,...,...,...
1651,Автошина R 17 215 / 60 Bridgestone Dueler H/T ...,215,60,17,96H,Dueler H/T D843,[Bridgestone],наши_данные,"O,O,B-radius,B-width,O,B-height,B-brand,B-line..."
1652,Автошина R 17 215 / 65 Bridgestone Blizzak LM0...,215,65,17,99H,Blizzak LM001,[Bridgestone],наши_данные,"O,O,B-radius,B-width,O,B-height,B-brand,B-line..."
1653,Шины R 17 225 / 50 Bridgestone Blizzak Ice 98T...,225,50,17,98T,Blizzak Ice,[Bridgestone],наши_данные,"O,O,B-radius,B-width,O,B-height,B-brand,B-line..."
1654,R 18 225 / 45 91S Bridgestone Blizzak VRX BRID...,225,45,18,91S,Blizzak VRX,"[Bridgestone, BRIDGESTONE]",наши_данные,"O,B-radius,B-width,O,B-height,O,B-brand,B-line..."


In [29]:
train_dataset

,offer,width,height,radius,v_ind,line,brand,type,BIO_Tags
0,17 / 205 / 50 Bridgestone Potenza RE050A I 89V...,205,50,17,89V,Potenza RE050A,[Bridgestone],наши_данные,"B-radius,O,B-width,O,B-height,B-brand,B-line,I..."
1,BRIDGESTONE 14823 Bridgestone 185 / 55 R 16 Ec...,185,55,16,83V,Ecopia EP300,"[BRIDGESTONE, Bridgestone]",наши_данные,"B-brand,O,B-brand,B-width,O,B-height,O,B-radiu..."
2,Bridgestone Blizzak LM005 155 / 65 R 14 79T зи...,155,65,14,79T,Blizzak LM005,[Bridgestone],наши_данные,"B-brand,B-line,I-line,B-width,O,B-height,O,B-r..."
3,Шины BRIDGESTONE Blizzak LM005 215 / 50 R 17 95 V,215,50,17,95V,Blizzak LM005,[BRIDGESTONE],наши_данные,"O,B-brand,B-line,I-line,B-width,O,B-height,O,B..."
4,Шины BRIDGESTONE Blizzak VRX 195 / 55 R 16 87S...,195,55,16,87S,Blizzak VRX,[BRIDGESTONE],наши_данные,"O,B-brand,B-line,I-line,B-width,O,B-height,O,B..."
...,...,...,...,...,...,...,...,...,...
6617,Bridgestone TURANZA T005 215 / 55 R 16 97W,215,55,16,97W,Turanza T005,[Bridgestone],наши_данные,"B-brand,O,B-line,B-width,O,B-height,O,B-radius,O"
6618,Bridgestone Blizzak Spike-02 215 / 55 R 16 93 T,215,55,16,93T,Blizzak Spike-02,[Bridgestone],наши_данные,"B-brand,B-line,I-line,B-width,O,B-height,O,B-r..."
6619,Шина 205 / 50 R 17 Bridgestone Blizzak LM005 93V,205,50,17,93V,Blizzak LM005,[Bridgestone],наши_данные,"O,B-width,O,B-height,O,B-radius,B-brand,B-line..."
6620,Зимние шины Bridgestone Blizzak Ice 195 / 55 R...,195,55,16,91T,Blizzak Ice,[Bridgestone],наши_данные,"O,O,B-brand,B-line,I-line,B-width,O,B-height,O..."


In [30]:
df_train_all = pd.concat([train_dataset, df_opensource])
df_train_all.to_csv('/home/sondors/Documents/price/BERT_NER/csv/prod_train/train_08our_1opensource.csv', sep=';',index=False)
df_train_all = df_train_all.reset_index(drop=True)
df_train_all

,offer,width,height,radius,v_ind,line,brand,type,BIO_Tags
0,17 / 205 / 50 Bridgestone Potenza RE050A I 89V...,205,50,17,89V,Potenza RE050A,[Bridgestone],наши_данные,"B-radius,O,B-width,O,B-height,B-brand,B-line,I..."
1,BRIDGESTONE 14823 Bridgestone 185 / 55 R 16 Ec...,185,55,16,83V,Ecopia EP300,"[BRIDGESTONE, Bridgestone]",наши_данные,"B-brand,O,B-brand,B-width,O,B-height,O,B-radiu..."
2,Bridgestone Blizzak LM005 155 / 65 R 14 79T зи...,155,65,14,79T,Blizzak LM005,[Bridgestone],наши_данные,"B-brand,B-line,I-line,B-width,O,B-height,O,B-r..."
3,Шины BRIDGESTONE Blizzak LM005 215 / 50 R 17 95 V,215,50,17,95V,Blizzak LM005,[BRIDGESTONE],наши_данные,"O,B-brand,B-line,I-line,B-width,O,B-height,O,B..."
4,Шины BRIDGESTONE Blizzak VRX 195 / 55 R 16 87S...,195,55,16,87S,Blizzak VRX,[BRIDGESTONE],наши_данные,"O,B-brand,B-line,I-line,B-width,O,B-height,O,B..."
...,...,...,...,...,...,...,...,...,...
12487,425 / 85 R 21 КАМА-1260-1 НК.ШЗ 14 146 J,425,85,21,NaN,КАМА-1260-1,НК.ШЗ,грузовая_резина,"B-width,O,B-height,O,B-radius,B-line,B-brand,O..."
12488,425 / 85 R 21 КАМА-1260-2 НК.ШЗ 18,425,85,21,NaN,КАМА-1260-2,НК.ШЗ,грузовая_резина,"B-width,O,B-height,O,B-radius,B-line,B-brand,O"
12489,425 / 85 R 21 О-184 TYREX CRG POWER Омск.ШЗ 14...,425,85,21,NaN,О-184 TYREX CRG POWER,Омск.ШЗ,грузовая_резина,"B-width,O,B-height,O,B-radius,B-line,I-line,I-..."
12490,425 / 85 R 21 О-184 TYREX CRG POWER Омск.ШЗ 18...,425,85,21,NaN,О-184 TYREX CRG POWER,Омск.ШЗ,грузовая_резина,"B-width,O,B-height,O,B-radius,B-line,I-line,I-..."


In [31]:
ilocc = df_train_all.iloc[12489]
print(ilocc.offer, '\n')
BIO = ilocc.BIO_Tags.split(',')
ofer = ilocc.offer.split()
for i in range(len(BIO)):
    print(f"{ofer[i]} --> {BIO[i]}")

425 / 85 R 21 О-184 TYREX CRG POWER Омск.ШЗ 14 146 K 

425 --> B-width
/ --> O
85 --> B-height
R --> O
21 --> B-radius
О-184 --> B-line
TYREX --> I-line
CRG --> I-line
POWER --> I-line
Омск.ШЗ --> B-brand
14 --> O
146 --> O
K --> O


In [32]:
test_dataset.to_csv('/home/sondors/Documents/price/BERT_NER/csv/prod_train/test_02our.csv', sep=';',index=False)


In [33]:
label2id = {'B-width': 1,
            'B-height': 2,
            'B-radius': 3,
            'B-brand': 4,
            'B-line': 5,
            'I-line': 6,
            'B-v_ind':7,
            'O': 0}
    
id2label = {1: 'B-width',
            2: 'B-height', 
            3: 'B-radius', 
            4: 'B-brand', 
            5: 'B-line', 
            6: 'I-line',
            7: 'B-v_ind',
            0: 'O'}